In [2]:
import numpy as np
import pandas as pd

In [3]:
customer_codes = pd.read_excel('Excel_report/Azimi.xlsx', sheet_name='Codes', index_col=0)
Azimi_report = pd.read_excel('Excel_report/Azimi.xlsx', sheet_name='Report', index_col=0)
receipt = pd.read_excel('Excel_report/Report.xlsm', sheet_name='Receipt', index_col=0)
delivery = pd.read_excel('Excel_report/Report.xlsm', sheet_name='Delivery', index_col=0)
# receipt = receipt.reset_index()
# delivery = delivery.reset_index()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: CollReport[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Table16[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: CollReport[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Table16[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [4]:
collective = pd.concat([receipt,delivery], axis=0)
collective = collective.sort_values('تاريخ میلادی')
collective = collective.reset_index()
collective[-10:]

,تاريخ,تاريخ میلادی,کد ردیابی,مقدار (کیلوگرم)
2647,00/10/04,2021-12-25,10018,422.0
2648,00/10/04,2021-12-25,10018,18.0
2649,00/10/04,2021-12-25,10104,320.0
2650,00/10/04,2021-12-25,10104,264.0
2651,00/10/04,2021-12-25,10020,2149.0
2652,00/10/04,2021-12-25,50016,-500.0
2653,00/10/05,2021-12-26,10119,104.0
2654,00/10/05,2021-12-26,10069,1580.0
2655,00/10/05,2021-12-26,10003,2686.0
2656,00/10/05,2021-12-26,30003,289.0


In [5]:
collective[collective['کد ردیابی']==10003].iloc[-10:]

,تاريخ,تاريخ میلادی,کد ردیابی,مقدار (کیلوگرم)
2529,00/09/24,2021-12-15,10003,2602.0
2559,00/09/27,2021-12-18,10003,-2990.0
2564,00/09/28,2021-12-19,10003,-2092.0
2581,00/09/29,2021-12-20,10003,2244.0
2585,00/09/29,2021-12-20,10003,-1396.0
2596,00/09/30,2021-12-21,10003,-2240.0
2617,00/10/01,2021-12-22,10003,-971.0
2627,00/10/01,2021-12-22,10003,2052.0
2637,00/10/03,2021-12-24,10003,7298.0
2655,00/10/05,2021-12-26,10003,2686.0


In [6]:
collective['بدهکار'] = -(collective['مقدار (کیلوگرم)'][collective['مقدار (کیلوگرم)']<0])
collective['بستانکار'] = collective['مقدار (کیلوگرم)'][collective['مقدار (کیلوگرم)']>0]
collective = collective.fillna(0)
collective['مانده بدهکار'] = collective.groupby('کد ردیابی')['بدهکار'].cumsum()
collective['مانده بستانکار'] = collective.groupby('کد ردیابی')['بستانکار'].cumsum()

In [7]:
collective['مانده در خط'] = collective['مانده بدهکار'] - collective['مانده بستانکار']

In [8]:
def not_settled_inline(collective):
    collective['مانده درخط تسویه نشده'] = collective['مانده بدهکار'] - collective['بستانکار'].sum()
    collective.loc[collective['مانده درخط تسویه نشده']<0,'مانده درخط تسویه نشده'] = 0
    return collective

collective = collective.groupby('کد ردیابی', group_keys=False).apply(not_settled_inline)

In [9]:
collective['مانده در خط تسویه شده'] = collective['مانده در خط'] - collective['مانده درخط تسویه نشده']

In [10]:
collective['سفارش تسویه شده'] = collective['بدهکار'] - collective['مانده درخط تسویه نشده']
collective.loc[collective['سفارش تسویه شده']<0,'سفارش تسویه شده'] = 0

In [11]:
collective['سفارش تسویه نشده'] = collective['بدهکار'] - collective['سفارش تسویه شده']

In [14]:
collective[collective['کد ردیابی']==10003].iloc[:,[0,1,2,4,5,8,11,12]][-15:]

,تاريخ,تاريخ میلادی,کد ردیابی,بدهکار,بستانکار,مانده در خط,سفارش تسویه شده,سفارش تسویه نشده
2499,00/09/22,2021-12-13,10003,0.0,1968.0,38519.0,0.0,0.0
2504,00/09/22,2021-12-13,10003,3107.0,0.0,41626.0,0.0,3107.0
2515,00/09/23,2021-12-14,10003,2194.0,0.0,43820.0,0.0,2194.0
2518,00/09/24,2021-12-15,10003,3030.0,0.0,46850.0,0.0,3030.0
2528,00/09/24,2021-12-15,10003,0.0,2457.0,44393.0,0.0,0.0
2529,00/09/24,2021-12-15,10003,0.0,2602.0,41791.0,0.0,0.0
2559,00/09/27,2021-12-18,10003,2990.0,0.0,44781.0,0.0,2990.0
2564,00/09/28,2021-12-19,10003,2092.0,0.0,46873.0,0.0,2092.0
2581,00/09/29,2021-12-20,10003,0.0,2244.0,44629.0,0.0,0.0
2585,00/09/29,2021-12-20,10003,1396.0,0.0,46025.0,0.0,1396.0


In [13]:
columns = pd.Series(collective.columns)
columns

0                     تاريخ
1              تاريخ میلادی
2                 کد ردیابی
3           مقدار (کیلوگرم)
4                    بدهکار
5                  بستانکار
6              مانده بدهکار
7            مانده بستانکار
8               مانده در خط
9     مانده درخط تسویه نشده
10    مانده در خط تسویه شده
11          سفارش تسویه شده
12         سفارش تسویه نشده
dtype: object